<a href="https://colab.research.google.com/github/Jaiquisic23/Artificial-Neural-Networks/blob/main/Jaime_Diamonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
''''
It is a regression task. The goal is to predict diamond price using the basic
diamond properties. Use price columns as labels. Handle categorical features.
Split the dataset into training and validation subset. Create training and
validation pipeline. Try to reach minimum cost value and prevent over-fitting.

Some hints:
1. The easiest way to load a csv file is using Pandas read_csv method

2. You can handle categorical features using scikit-learn estimators:
OrdinalEncoder or OneHotEncoder.

3. Columns have different scales normalize them before training. For this
purpose you can use Tensorflow LayerNormalization layer.

3. It is a regression task so be aware of correct model architecture and cost
function. Typical regressions cost functions are: MAE, MSE. If you want to
predict output in range(0, 1000) then you can not use sigmoid activation
function in output layer.
'''

"'\nIt is a regression task. The goal is to predict diamond price using the basic\ndiamond properties. Use price columns as labels. Handle categorical features.\nSplit the dataset into training and validation subset. Create training and\nvalidation pipeline. Try to reach minimum cost value and prevent over-fitting.\n\nSome hints:\n1. The easiest way to load a csv file is using Pandas read_csv method\n\n2. You can handle categorical features using scikit-learn estimators:\nOrdinalEncoder or OneHotEncoder.\n\n3. Columns have different scales normalize them before training. For this\npurpose you can use Tensorflow LayerNormalization layer.\n\n3. It is a regression task so be aware of correct model architecture and cost\nfunction. Typical regressions cost functions are: MAE, MSE. If you want to\npredict output in range(0, 1000) then you can not use sigmoid activation\nfunction in output layer.\n"

In [11]:
from google.colab import files
dataload = files.upload()

import pandas as pd
import numpy as np
import io
import seaborn as sns
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from math import sqrt


from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso






Saving diamonds.csv to diamonds (3).csv


In [12]:
dataload = pd.read_csv(io.StringIO(dataload['diamonds.csv'].decode('utf-8')))

In [13]:
dataload.head(10)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,6,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,7,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,8,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,9,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,10,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


In [14]:
print('Shape:', dataload.shape)

Shape: (53940, 11)


In [15]:
dataload = dataload.drop(["Unnamed: 0"], axis=1)
dataload.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [18]:
dataload = dataload.drop(dataload[dataload["x"]==0].index)
dataload = dataload.drop(dataload[dataload["y"]==0].index)
dataload = dataload.drop(dataload[dataload["z"]==0].index)


In [21]:
print('Dataload after removing incorrect data')      #Every 0 in the line "min" means that there are diamonds without dimensions, so we eliminate them
dataload.describe(include = 'all')

Dataload after removing incorrect data


,carat,cut,color,clarity,depth,table,price,x,y,z
count,53920.000000,53920,53920,53920,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000
unique,NaN,5,7,8,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Ideal,G,SI1,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,21548,11284,13063,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.797698,NaN,NaN,NaN,61.749514,57.456834,3930.993231,5.731627,5.734887,3.540046
std,0.473795,NaN,NaN,NaN,1.432331,2.234064,3987.280446,1.119423,1.140126,0.702530
min,0.200000,NaN,NaN,NaN,43.000000,43.000000,326.000000,3.730000,3.680000,1.070000
25%,0.400000,NaN,NaN,NaN,61.000000,56.000000,949.000000,4.710000,4.720000,2.910000
50%,0.700000,NaN,NaN,NaN,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,NaN,NaN,NaN,62.500000,59.000000,5323.250000,6.540000,6.540000,4.040000


In [ ]:
sns.pairplot(dataload, hue = 'cut')